- Glow WGR
    - [Ridge Models](https://github.com/projectglow/glow/blob/f3edf5bb8fe9c2d2e1a374d4402032ba5ce08e29/python/glow/wgr/linear_model/ridge_model.py#L243)
        - RidgeReducer and RidgeRegressor
    - [Ridge UDFs](https://github.com/projectglow/glow/blob/f3edf5bb8fe9c2d2e1a374d4402032ba5ce08e29/python/glow/wgr/linear_model/ridge_udfs.py#L420)
        - [map_normal_eqn](https://github.com/projectglow/glow/blob/f3edf5bb8fe9c2d2e1a374d4402032ba5ce08e29/python/glow/wgr/linear_model/ridge_udfs.py#L67)
        - [reduce_normal_eqn](https://github.com/projectglow/glow/blob/f3edf5bb8fe9c2d2e1a374d4402032ba5ce08e29/python/glow/wgr/linear_model/ridge_udfs.py#L150)
    - [Functions](https://github.com/projectglow/glow/blob/f3edf5bb8fe9c2d2e1a374d4402032ba5ce08e29/python/glow/wgr/linear_model/functions.py#L107)
        - [assemble_block](https://github.com/projectglow/glow/blob/f3edf5bb8fe9c2d2e1a374d4402032ba5ce08e29/python/glow/wgr/linear_model/functions.py#L78)
        - [evaluate_coefficients](https://github.com/projectglow/glow/blob/f3edf5bb8fe9c2d2e1a374d4402032ba5ce08e29/python/glow/wgr/linear_model/functions.py#L135)
    - block_variants_and_samples
        - [blockvariantsandsamples/VariantSampleBlockMaker.scala](https://github.com/projectglow/glow/blob/054c3f4c714cf1693c964554842bf3ba67d76eb4/core/src/main/scala/io/projectglow/transformers/blockvariantsandsamples/VariantSampleBlockMaker.scala)
        - [functions.py/block_variants_and_samples](https://github.com/projectglow/glow/blob/04257f65ad64b45b2ad4a9417292e0ead6f94212/python/glow/wgr/functions.py#L91)
    - Alpha values: 
        - https://glow.readthedocs.io/en/latest/tertiary/whole-genome-regression.html#id4            
        - https://github.com/projectglow/glow/blob/f3edf5bb8fe9c2d2e1a374d4402032ba5ce08e29/python/glow/wgr/linear_model/ridge_model.py#L189
        - https://github.com/projectglow/glow/blob/f3edf5bb8fe9c2d2e1a374d4402032ba5ce08e29/python/glow/wgr/linear_model/functions.py#L260
            - These are number of columns divided by heritability constants 
        - Default alphas are the same at both levels (but based on number of features)
    - Scaling
        - Covariates and phenotypes are scaled externally in the example notebook, and variants are scaled in block_variants_and_samples:
            - https://glow.readthedocs.io/en/latest/_static/notebooks/tertiary/glowgr.html
            - The 'block_variants_and_samples' output seems to indicate that the variant mean/sd is across whole dataset (it could be per sample block, but that wouldn't make much sense)
        - Precomputed mu/sig is applied blockwise here, though this appears equivalent to doing it globally:
            - https://github.com/projectglow/glow/blob/f3edf5bb8fe9c2d2e1a374d4402032ba5ce08e29/python/glow/wgr/linear_model/functions.py#L107
            - Note that this same logic is applied to both genotypes and predictions from the first regression stage
    - Covariates are used at both levels, see: https://glow.readthedocs.io/en/latest/tertiary/whole-genome-regression.html notebook at bottom 
    - Scoring / Model Selection
        - In stage 2 ridge regression, alphas chosen based on mean r2 value, ties broken by larger alpha
            - https://github.com/projectglow/glow/blob/f3edf5bb8fe9c2d2e1a374d4402032ba5ce08e29/python/glow/wgr/linear_model/ridge_model.py#L243
            - Actual scoring logic:
                - https://github.com/projectglow/glow/blob/f3edf5bb8fe9c2d2e1a374d4402032ba5ce08e29/python/glow/wgr/linear_model/ridge_udfs.py#L420
                - r2 definition: https://github.com/projectglow/glow/blob/f3edf5bb8fe9c2d2e1a374d4402032ba5ce08e29/python/glow/wgr/linear_model/functions.py#L227
        - The scoring works by calculating R2 for each outcome/phenotype and **sample block** (across all variants come features in stage 1), taking the average of these scores, and then making "transform" choose the predictions for the model with an alpha correspond to that R2 across all folds
            - https://github.com/projectglow/glow/blob/f3edf5bb8fe9c2d2e1a374d4402032ba5ce08e29/python/glow/wgr/linear_model/ridge_model.py#L247
- REGENIE Methods
    - Missingness
        - Traits are fit separately in stage 2 since the predictions that are now covariates are specific to one outcome:
            - "Since the block predictions will differ across traits, we then fit the model in Eq(16) separately for each trait."
        - This means that it is easy to discard missing samples after stage 1 where covariates are the same and multiple phenotypes are solved for simulatenously
- Ridge Regression
    - ridge eq: https://stats.stackexchange.com/questions/69205/how-to-derive-the-ridge-regression-solution
    - sklearn [svd ridge](https://github.com/scikit-learn/scikit-learn/blob/fd237278e895b42abe8d8d09105cbb82dc2cbba7/sklearn/linear_model/_ridge.py#L220)
    - sklearn [ridge solver switch for cholesky and svd](https://github.com/scikit-learn/scikit-learn/blob/fd237278e/sklearn/linear_model/_ridge.py#L238)
        - The cholesky solver supports sparse data while svd does not
        - SVD works with singular matrices though
- Least squares/ridge solutions
    - Cholesky is generally faster than SVD but less stable with near-singular matrices
    - Glow is using vanilla matrix inverse for ridge solution?!: https://github.com/projectglow/glow/blob/f3edf5bb8fe9c2d2e1a374d4402032ba5ce08e29/python/glow/wgr/linear_model/functions.py#L154
    - scikit-learn uses lstsq in LinearRegression
        - See: https://github.com/scikit-learn/scikit-learn/blob/fd237278e/sklearn/linear_model/_base.py#L546
        - numpy lstsq uses LAPACK routine `dgelsd`, which uses SVD
            - See: https://stats.stackexchange.com/questions/240573/how-does-numpy-solve-least-squares-for-underdetermined-systems
    - dask lstsq uses QR decomp rather than SVD like numpy
    - It is possible to solve a system of linear equations given a covariance matrix (covariance between features, not samples) rather than a data matrix
        - https://stats.stackexchange.com/questions/107597/is-there-a-way-to-use-the-covariance-matrix-to-find-coefficients-for-multiple-re
- SVD
    - Dask SVD is 2D only
        - https://github.com/dask/dask/blob/master/dask/array/linalg.py#L687 (svd_compressed)
        - https://github.com/dask/dask/blob/master/dask/array/linalg.py#L786 (svd)
            - This uses QR underneath, just like lstsq

In [1]:
import numpy as np
import pandas as pd

#### Equivalences

Show some equality between a few implementations.

Notes:

- Ridge closed-form univariate regressions are equal to multivariate versions (within sklearn and as matrix ops)
  - https://stats.stackexchange.com/questions/318810/multivariate-linear-regression-vs-several-univariate-regression-models
- Adding orthogonalization of genotypes/phenotypes using covariates does NOT give the same solution as ridge w/ all features
  - Even when the residuals are based on regularized OLS solutions using same alphas, it's not the same as including all features
      - It does appear that doing this is less different than creating the residuals without alpha

In [299]:
m, n, c, y = 10, 25, 2, 3
np.random.seed(0)
X = np.random.normal(size=(n, c))
BX = np.random.normal(size=(X.shape[1], 1)) 
G = np.random.choice([0, 1, 2], size=(m, n))
BG = np.random.normal(size=(m, y))

Y = X @ BX + G.T @ BG + np.random.normal(size=(n, y), scale=.001)
Y[:10]

array([[-4.88819927e+00, -5.64811555e+00, -3.62265486e+00],
       [-5.20096222e-03, -9.74021294e-01, -2.68771516e+00],
       [-3.43436956e+00,  1.55021483e+00, -5.90893130e+00],
       [-4.11500059e+00, -4.80200099e+00, -8.00024795e-01],
       [-1.93529933e+00,  5.64037809e-01, -8.67861009e-01],
       [-2.85120037e+00,  3.15536321e-01, -2.19386007e+00],
       [-6.36352718e+00, -5.57324266e+00, -4.48401686e+00],
       [-1.79675449e+00, -2.53097560e+00, -2.94050474e-01],
       [-6.02852585e+00, -4.13910764e+00, -1.75846828e+00],
       [ 1.89095768e+00,  2.17062373e+00, -2.31960025e+00]])

In [300]:
X.shape, G.shape, Y.shape

((25, 2), (10, 25), (25, 3))

In [208]:
from sklearn.linear_model import Ridge, LinearRegression


def _solve_svd(X, y, alpha):
    # From: https://github.com/scikit-learn/scikit-learn/blob/fd237278e895b42abe8d8d09105cbb82dc2cbba7/sklearn/linear_model/_ridge.py#L220
    # Note: alphas must be either of size 1 or n_targets
    U, s, Vt = np.linalg.svd(X, full_matrices=False)
    idx = s > 1e-15  # same default value as scipy.linalg.pinv
    s_nnz = s[idx][:, np.newaxis]
    UTy = np.dot(U.T, y)
    d = np.zeros((s.size, alpha.size), dtype=X.dtype)
    d[idx] = s_nnz / (s_nnz ** 2 + alpha)
    d_UT_y = d * UTy
    return np.dot(Vt.T, d_UT_y).T


def _ridge_regression_inv(X, Y, alpha):
    XtX = X.T @ X
    XtY = X.T @ Y
    B = np.linalg.inv(XtX + np.eye(X.shape[1]) * alpha) @ XtY
    return B.T
    
    
def _ridge_regression_svd(X, Y, alpha):
    B = _solve_svd(X, Y, np.array([alpha]))
    return B


def ridge_regression(G, X, Y, alpha, ortho=None):
    if ortho:
        # G = G - X @ (np.linalg.inv(X.T @ X) @ (X.T @ G))
        # Y = Y - X @ (np.linalg.inv(X.T @ X) @ (X.T @ Y))
        if ortho == 'ridge':
            G = G - X @ _ridge_regression_svd(X, G, alpha).T
            Y = Y - X @ _ridge_regression_svd(X, Y, alpha).T
        elif ortho == 'lstsq':
            G = G - X @ np.linalg.lstsq(X, G, rcond=None)[0]
            Y = Y - X @ np.linalg.lstsq(X, Y, rcond=None)[0]
        else:
            raise ValueError
        return _ridge_regression_svd(G, Y, alpha).T
    else:
        X = np.concatenate((X, G), axis=1)
        return _ridge_regression_svd(X, Y, alpha).T


def test(G, X, Y, alpha=.01, scale=False):
    coefs = []
    if scale:
        G = (G - G.mean(axis=0)) / G.std(axis=0)
        X = (X - X.mean(axis=0)) / X.std(axis=0)
        Y = (Y - Y.mean(axis=0)) / Y.std(axis=0)
    
    xn = [f'x{i:03d}' for i in range(X.shape[1])]
    gn = [f'g{i:03d}' for i in range(G.shape[1])]
    
    # Add intercept
    X = np.concatenate((np.ones((X.shape[0], 1)), X), axis=1)
    XG = np.concatenate((X, G), axis=1)
    
    B = ridge_regression(G, X, Y, alpha=alpha)
    coefs.append(pd.DataFrame(B, index=['i'] + xn + gn).assign(src='ridge_ortho=None'))
    
    B = ridge_regression(G, X, Y, alpha=alpha, ortho='lstsq')
    coefs.append(pd.DataFrame(B, index=gn).assign(src='ridge_ortho=lstsq'))
    
    B = ridge_regression(G, X, Y, alpha=alpha, ortho='ridge')
    coefs.append(pd.DataFrame(B, index=gn).assign(src='ridge_ortho=ridge'))
    
    # This will return the same coefficients for everything except the 
    # intercept when comparing cholesky and svd solvers
    est = Ridge(alpha=alpha, random_state=0, fit_intercept=False, solver='svd').fit(XG, Y)
    coefs.append(pd.DataFrame(est.coef_.T, index=['i'] + xn + gn).assign(src='skridge'))
    
    B = np.stack([
        Ridge(alpha=alpha, random_state=0, fit_intercept=False, solver='svd').fit(XG, Y[:, i]).coef_
        for i in range(Y.shape[1])
    ], axis=-1)
    coefs.append(pd.DataFrame(B, index=['i'] + xn + gn).assign(src='skridge-univ'))
    
    ols = LinearRegression(fit_intercept=False)
    ols = ols.fit(XG, Y)
    coefs.append(pd.DataFrame(ols.coef_.T, index=['i'] + xn + gn).assign(src='ols'))
    
    B = np.stack([
        LinearRegression(fit_intercept=False).fit(XG, Y[:, i]).coef_
        for i in range(Y.shape[1])
    ], axis=-1)
    coefs.append(pd.DataFrame(B, index=['i'] + xn + gn).assign(src='ols-univ'))
    
    df = pd.concat(coefs)
    df.index.name = 'feat'
    df = df.reset_index().set_index(['src', 'feat'])
    df = df.rename(columns=lambda c: f'Y{c:03d}')
    df.columns.name = 'outcome'
    df = df.stack().rename('value').reset_index()
    return df.pivot_table(index='feat', columns=['src', 'outcome'], values='value')


# Show that OLS and ridge are equal for alpha 0
df = test(G.T, X, Y, alpha=0, scale=False)
np.testing.assert_allclose(df['ols'], df['ridge_ortho=None'], atol=1e-14) 
np.testing.assert_allclose(df['ols-univ'], df['ridge_ortho=None'], atol=1e-14) 
np.testing.assert_allclose(df['ridge_ortho=None'], df['skridge'], atol=1e-14)
np.testing.assert_allclose(df['ridge_ortho=None'].loc[df['ridge_ortho=lstsq'].dropna().index], df['ridge_ortho=lstsq'].dropna(), atol=1e-14)
np.testing.assert_allclose(df['ridge_ortho=None'].loc[df['ridge_ortho=ridge'].dropna().index], df['ridge_ortho=ridge'].dropna(), atol=1e-14)


# Show that ridge methods are equal w and wo scaling
df = test(G.T, X, Y, alpha=100, scale=False)
np.testing.assert_allclose(df['ridge_ortho=None'], df['skridge'])
np.testing.assert_allclose(df['ridge_ortho=None'], df['skridge-univ'])
df = test(G.T, X, Y, alpha=100, scale=True)
np.testing.assert_allclose(df['ridge_ortho=None'], df['skridge'], atol=1e-14)
np.testing.assert_allclose(df['ridge_ortho=None'], df['skridge-univ'], atol=1e-14)

In [209]:
df.head()

src           ols                      ols-univ                      \
outcome      Y000      Y001      Y002      Y000      Y001      Y002   
feat                                                                  
g000    -0.090064 -0.388859 -0.210753 -0.090064 -0.388859 -0.210753   
g001    -0.653722  0.204154 -0.605877 -0.653722  0.204154 -0.605877   
g002    -0.303247  0.016031 -0.261491 -0.303247  0.016031 -0.261491   
g003     0.326126 -0.303825  0.072497  0.326126 -0.303825  0.072497   
g004    -0.010723 -0.348822  0.180845 -0.010723 -0.348822  0.180845   

src     ridge_ortho=None                     ridge_ortho=lstsq  ...            \
outcome             Y000      Y001      Y002              Y000  ...      Y002   
feat                                                            ...             
g000           -0.048020 -0.069805 -0.033972         -0.054608  ... -0.040013   
g001           -0.107376 -0.003692 -0.126415         -0.079918  ... -0.081279   
g002           -0.055718 -0.008892 -0.046376         -0.041848  ... -0.031620   
g003           -0.005775 -0.059849 -0.035594          0.009806  ... -0.007506   
g004           -0.048311 -0.064184  0.021058         -0.035193  ...  0.042037   

src     ridge_ortho=ridge                       skridge                      \
outcome              Y000      Y001      Y002      Y000      Y001      Y002   
feat                                                                          
g000            -0.049062 -0.068822 -0.034760 -0.048020 -0.069805 -0.033972   
g001            -0.102339  0.000943 -0.118038 -0.107376 -0.003692 -0.126415   
g002            -0.053454 -0.010070 -0.044198 -0.055718 -0.008892 -0.046376   
g003            -0.002831 -0.056136 -0.030209 -0.005775 -0.059849 -0.035594   
g004            -0.045925 -0.062226  0.024911 -0.048311 -0.064184  0.021058   

src     skridge-univ                      
outcome         Y000      Y001      Y002  
feat                                      
g000       -0.048020 -0.069805 -0.033972  
g001       -0.107376 -0.003692 -0.126415  
g002       -0.055718 -0.008892 -0.046376  
g003       -0.005775 -0.059849 -0.035594  
g004       -0.048311 -0.064184  0.021058  

[5 rows x 21 columns]

In [210]:
sorted(set([c[0] for c in df.columns]))

['ols',
 'ols-univ',
 'ridge_ortho=None',
 'ridge_ortho=lstsq',
 'ridge_ortho=ridge',
 'skridge',
 'skridge-univ']

In [211]:
df['ridge_ortho=ridge'] - df['ridge_ortho=lstsq']

outcome,Y000,Y001,Y002
feat,,,
g000,0.005546,-0.003049,0.005254
g001,-0.022421,-0.019546,-0.036759
g002,-0.011605,0.003124,-0.012578
g003,-0.012637,-0.015092,-0.022702
g004,-0.010732,-0.008395,-0.017127
g005,0.007650,0.004267,0.011372
g006,-0.010752,-0.011892,-0.018855
g007,0.008859,0.012995,0.017092
g008,-0.005591,-0.005343,-0.009394


In [212]:
(df['skridge'] - df['ridge_ortho=lstsq']).dropna().describe()

outcome,Y000,Y001,Y002
count,10.000000,10.000000,10.000000
mean,-0.005692,-0.004131,-0.008927
std,0.013124,0.013338,0.021660
min,-0.027458,-0.024180,-0.045135
25%,-0.013711,-0.013685,-0.022707
50%,-0.009988,-0.005329,-0.013158
75%,0.006522,0.004975,0.011555
max,0.010986,0.016251,0.021261


In [213]:
(df['skridge'] - df['ridge_ortho=ridge']).dropna().describe()

outcome,Y000,Y001,Y002
count,10.000000,10.000000,10.000000
mean,-0.001029,-0.000773,-0.001627
std,0.002403,0.002639,0.004054
min,-0.005037,-0.004634,-0.008377
25%,-0.002457,-0.002667,-0.004284
50%,-0.001766,-0.001133,-0.002171
75%,0.001209,0.001116,0.002047
max,0.002127,0.003255,0.004169


### ND-Extension

Extend ridge closed form SVD solution to include alphas.

In [290]:
def _solve_svd_single(X, y, alpha):
    # From: https://github.com/scikit-learn/scikit-learn/blob/fd237278e895b42abe8d8d09105cbb82dc2cbba7/sklearn/linear_model/_ridge.py#L220
    # Note: alphas must be either of size 1 or n_targets
    U, s, Vt = np.linalg.svd(X, full_matrices=False)
    idx = s > 1e-15  # same default value as scipy.linalg.pinv
    s_nnz = s[idx][:, np.newaxis]
    UTy = np.dot(U.T, y)
    d = np.zeros((s.size, alpha.size), dtype=X.dtype)
    d[idx] = s_nnz / (s_nnz ** 2 + alpha)
    d_UT_y = d * UTy
    return np.dot(Vt.T, d_UT_y).T


def _solve_svd_single2(X, y, alpha):
    # Sames as above but with no indexer on non-zero singular values
    # Note: alphas must be either of size 1 or n_targets
    U, s, Vt = np.linalg.svd(X, full_matrices=False)
    s = np.where(s > 1e-15, s, 0.0)
    s = s[:, np.newaxis]
    #idx = s > 1e-15  # same default value as scipy.linalg.pinv
    #s_nnz = s[idx][:, np.newaxis]
    UTy = np.dot(U.T, y)
    d = np.zeros((s.size, alpha.size), dtype=X.dtype)
    d = s / (s ** 2 + alpha)
    d_UT_y = d * UTy
    return np.dot(Vt.T, d_UT_y).T

def _solve_svd_multi(X, Y, a):
    # From: https://github.com/scikit-learn/scikit-learn/blob/fd237278e895b42abe8d8d09105cbb82dc2cbba7/sklearn/linear_model/_ridge.py#L220
    # Note: alphas should all be different (each one applied to each outcome)
    # nc = number of columns in X (features)
    # ns = number of singular values in X (min of cols/rows)
    # na = number of alphas
    # ny = number of outcomes
    U, s, Vt = np.linalg.svd(X, full_matrices=False)
    # Clip below default threshold (from scipy.linalg.pinv)
    # s = np.where(s > 1e-15, s, 0.0) # Do this for scikit-learn equivalence
    UtY = np.dot(U.T, Y) # shape=(ns, ny)
    Vtt = Vt.T # shape=(nc, ns), nc = X.shape[1]
    
    # Expand singular values and alphas so that broadcasting
    # generates (a.size, s.size), i.e. (na, ns), arrays where s.size is 
    # number of columns/rows in X (whichever is smaller)
    s = np.expand_dims(s, 0)
    a = np.expand_dims(a, -1)
    
    # Compute normalization and expand to (na, ns, 1)
    # for broadcast w/ UtY of shape (ns, ny)
    # to generate array of shape (na, ns, ny)
    d = np.expand_dims(s / (s ** 2 + a), -1)
    d_UtY = d * UtY
    
    # Compute stacked multiplication with alphas in dim 0
    # so that trailing Vtt (nc, ns) and d_UtY (ns, ny)
    # multiplications are performed as w/ single alpha
    # Returns: (na, nc, ny)
    print(s.shape, d.shape, UtY.shape, Vt.T.shape, d_UtY.shape)
    return np.matmul(np.expand_dims(Vtt, 0), d_UtY)

In [295]:
np.random.seed(0)
X = np.random.normal(size=(100, 12))
B = np.random.normal(size=(12, 6))
Y = X @ B + np.random.normal(scale=.001)
Y.shape

(100, 6)

In [297]:
alphas = np.array([.1, .2, 1., 10., 0.])
B = _solve_svd_multi(X, Y, alphas)
B.shape

(1, 12) (5, 12, 1) (12, 6) (12, 12) (5, 12, 6)


(5, 12, 6)

In [298]:
for i, a in enumerate(alphas):
    np.testing.assert_allclose(B[i].T, _solve_svd_single(X, Y, np.array([a])))

In [249]:
np.testing.assert_allclose(
    _solve_svd_single(X, Y, np.array([.1])),
    _solve_svd_single2(X, Y, np.array([.1]))
)